# Imports

## Packages

In [1]:
import torch.optim
from torch.utils import data
import argparse
import json
from tqdm import tqdm
import sys
import clip
sys.path.append('/home/guest/Documents/Siraj TM/RSCaMa')
from model.model_encoder_attMamba import Encoder, AttentiveEncoder
from model.model_decoder import DecoderTransformer
from utils_tool.utils import *
from data.LEVIR_CC.LEVIRCC import LEVIRCCDataset
from torch import nn, einsum

from PIL import Image
from imageio import imread
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import time

/home/guest/anaconda3/envs/RSCaMa_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.device_count()

2

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda:1")

In [4]:
!nvidia-smi

Mon Mar 31 16:24:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:65:00.0 Off |                  Off |
| 30%   42C    P8             11W /  230W |   11142MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Functions

In [5]:
def load_json(path):
    with open(path) as f:
        file = json.load(f)
    f.close()
    return file
def save_json(file,path):
    with open(path,'w') as f:
        json.dump(file,f)
    f.close()
    print("Saved Successfully")
def rem_print(word):
    t_word = word
    for _ in range(100 - len(t_word)):
        word = word + ' '
    print(word,end='\r')

## Data

In [6]:
word_vocab = load_json('assets/vocab.json')

In [7]:
train_dir = 'data/Levir-CC-dataset/images/train'

# Training

## Imports

In [8]:
from torch.utils import data
from torch.nn.utils.rnn import pack_padded_sequence

## Functions

## Data

In [9]:
train_loader = data.DataLoader(
                LEVIRCCDataset('CLIP-ViT-B/32','data/Levir-CC-dataset/images', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/', 'train', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/tokens/', word_vocab, 42, 1),
                batch_size=64, shuffle=True, num_workers=24, pin_memory=True)

In [10]:
val_loader = data.DataLoader(
                LEVIRCCDataset('CLIP-ViT-B/32','data/Levir-CC-dataset/images', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/', 'val', '/home/guest/Documents/Siraj TM/RSCaMa/data/LEVIR_CC/tokens/', word_vocab , 42, 1),
                batch_size=64, shuffle=False, num_workers=24, pin_memory=True)

## Model

In [11]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [12]:
encoder = Encoder('CLIP-ViT-B/32')
encoder.fine_tune(True)
encoder_trans = AttentiveEncoder(n_layers=3,
                                        feature_size=[7, 7, 768],
                                        heads=8, dropout=0.1)
decoder = DecoderTransformer(decoder_type='transformer_decoder',embed_dim=768,
                                    vocab_size=len(word_vocab), max_lengths=42,
                                    word_vocab=word_vocab, n_head=8,
                                    n_layers=1, dropout=0.1)

decoder_n_layers= 1
decoder_type= transformer_decoder


In [20]:
!nvidia-smi

Mon Mar 31 16:25:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:65:00.0 Off |                  Off |
| 30%   42C    P8             11W /  230W |   11142MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
num_epochs = 25

In [16]:
encoder_optimizer = torch.optim.Adam(params=encoder.parameters(),
                                            lr=1e-4) if True else None
encoder_trans_optimizer = torch.optim.Adam(
    params=filter(lambda p: p.requires_grad, encoder_trans.parameters()),
    lr=1e-4)
decoder_optimizer = torch.optim.Adam(
    params=filter(lambda p: p.requires_grad, decoder.parameters()),
    lr=1e-4)

# Move to GPU, if available
encoder.cuda(device)
encoder_trans.cuda(device)
decoder.cuda(device)
encoder_lr_scheduler = torch.optim.lr_scheduler.StepLR(encoder_optimizer, step_size=5,
                                                            gamma=1.0) if True else None
encoder_trans_lr_scheduler = torch.optim.lr_scheduler.StepLR(encoder_trans_optimizer, step_size=5,
                                                                    gamma=1.0)
decoder_lr_scheduler = torch.optim.lr_scheduler.StepLR(decoder_optimizer, step_size=5,
                                                            gamma=1.0)
hist = np.zeros((num_epochs*2 * len(train_loader), 5))

l_resizeA = torch.nn.Upsample(size = (256, 256), mode ='bilinear', align_corners = True)
l_resizeB = torch.nn.Upsample(size = (256, 256), mode ='bilinear', align_corners = True)
index_i = 0

## Training Step

In [19]:
encoder.train()
encoder_trans.train()
decoder.train()
criterion_cap = torch.nn.CrossEntropyLoss().cuda(device)
criterion_cap_cls = torch.nn.CrossEntropyLoss().cuda(device)

In [ ]:


print_freq = 100
EPOCHS = num_epochs

for epoch in range(EPOCHS):
    for id,batch_data in enumerate(train_loader):
        
        start_time = time.time()
        accum_steps = 64//64
        
        #Getting Data and moving to GPU if possible
        imgA = batch_data['imgA']
        imgB = batch_data['imgB']
        token = batch_data['token']
        token_len = batch_data['token_len']
        imgA = imgA.cuda(device)
        imgB = imgB.cuda(device)
        token = token.cuda(device)
        token_len = token_len.cuda(device)
        
        #Feat1 and Feat2
        feat1, feat2 = encoder(imgA, imgB)
        feat = encoder_trans(feat1, feat2)
        scores, caps_sorted, decode_lengths, sort_ind = decoder(feat, token, token_len)
        # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
        targets = caps_sorted[:, 1:]
        
        scores = pack_padded_sequence(scores, decode_lengths, batch_first=True).data
        targets = pack_padded_sequence(targets, decode_lengths, batch_first=True).data
        
        loss = criterion_cap(scores, targets.to(torch.int64))
        
        loss = loss / accum_steps
        loss.backward()
        
        if (id + 1) % accum_steps == 0 or (id + 1) == len(train_loader):
            decoder_optimizer.step()
            encoder_trans_optimizer.step()
            if encoder_optimizer is not None:
                encoder_optimizer.step()

            # Adjust learning rate
            decoder_lr_scheduler.step()
            encoder_trans_lr_scheduler.step()
            if encoder_lr_scheduler is not None:
                encoder_lr_scheduler.step()

            decoder_optimizer.zero_grad()
            encoder_trans_optimizer.zero_grad()
            if encoder_optimizer is not None:
                encoder_optimizer.zero_grad()
                
        hist[index_i, 0] = time.time() - start_time #batch_time
        hist[index_i, 1] = loss.item()  # train_loss
        hist[index_i, 2] = accuracy_v0(scores, targets, 5) #top5
        
        index_i += 1
        
        if index_i % (print_freq/10) == 0:
            rem_print(f'Training Epoch: {epoch}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

: 

In [1]:
!nvidia-smi

Mon Mar 31 16:17:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:65:00.0 Off |                  Off |
| 30%   44C    P8             10W /  230W |   11142MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----